In [2]:
###########import packages##########
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.backend.tensorflow_backend import set_session
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.models import Model
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import GridSearchCV
from sko.GA import GA
from sko.PSO import PSO
import time
from keras.models import Model, load_model
from sko.GA import GA
from sko.DE import DE
from sklearn import tree
%matplotlib
###########assign memory##########
###########delete this part if your tensorflow was based on CPU##########
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
config.gpu_options.per_process_gpu_memory_fraction = 0.45
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config)) 
###########fix random seed for reproducability##########
seed=1
np.random.seed(seed)
###########wrapping root mean square error for later calls##########
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
###########loading data##########
fdata=pd.read_csv('database.csv',encoding="gbk")
raw_data=fdata.loc[:,[                     
                      'A_c',#0
                      'A_a',#1
                      'L_cl',#2
                      'i0',#3
                      'L-PEM',#4
                      'L_GDL',#5
                      'epsl_cl',#6
                      'epsp_cl',#7
                      'T_cc',#8
                      'RH_cc',#9
                      'pressure',#10
                        'Volt0.9',
                        'Volt0.8',
                        'Volt0.7',#13
                        'Volt0.6',
                        'Volt0.5',
                        'Volt0.4',
                        'Volt0.3',
                        'Volt0.2',
                        'Volt0.1',
                        'Volt0'
                        ]]
###########data standardization##########
standardized_data = (raw_data-np.mean(raw_data,axis=0))/np.std(raw_data,axis=0)

###########defining a wrapper function for later call from each machine learning algorithms##########
raw_input=standardized_data.iloc[:,0:11]
raw_output=standardized_data.iloc[:,13]
X=raw_input.values.astype(np.float32)
y=raw_output.values.astype(np.float32)
###########fix random seed for reproducability##########
seed=1
###########train test splitting##########
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15,random_state=seed)
raw_input_global=raw_data.iloc[:,0:11]
raw_output_global=raw_data.iloc[:,13]
###########wrap up fuction for later call for OPTIMIZATION##########
def evaluate(pre_2,real_2):
    pre_2=np.array(pre_2)
    real_2=np.array(real_2)
    pre_2_series=pd.Series(pre_2)
    real_2_series=pd.Series(real_2)
    return rmse(pre_2,real_2), round(pre_2_series.corr(real_2_series), 3)
def compare(list_name,limit):
    judge=1
    for a in list_name:
        if a < limit:
            judge=judge*1
        else:
            judge=judge*0
    return judge
def generate_arrays_from_file(path):
    while True:
        with open(path) as f:
            for line in f:
                # create numpy arrays of input data
                # and labels, from each line in the file
                x1, x2, y = process_line(line)
                yield ({'input_1': x1, 'input_2': x2}, {'output': y})
for neurons1 in [50,250,500]:
    for epochs_number in [50,250,500]:
        for learning_rate_search in [0.000005,0.00005,0.0005]:
            regularizer=keras.regularizers.l2(0)
            dropout_rate=0
            batch_size_number=32
            activation1='relu'
            ###########keras ANN model construction##########
            model = Sequential() 
            model.add(Dense(neurons1, input_dim=11, kernel_initializer='random_normal',
                            bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
            model.add(Dropout(dropout_rate))
            model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                            bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
            model.add(Dropout(dropout_rate))
            model.add(Dense(1, input_dim=neurons1, activation='linear'))
            adam=optimizers.Adam(lr=learning_rate_search)
            model.compile(loss='mse', optimizer=adam)
            print('training...')
            model.fit(X_train, y_train,verbose=1, batch_size=batch_size_number,epochs=epochs_number)
            result=model.predict(X_test)
            ###########get RMSE and R2##########
            x_prediction_07=result*np.std(raw_output_global,axis=0)+np.mean(raw_output_global,axis=0)
            y_real_07=np.std(raw_output_global,axis=0)*y_test+np.mean(raw_output_global,axis=0)
            print(x_prediction_07[:,0])
            print(y_real_07)
            x_prediction_07_series=pd.Series(x_prediction_07[:,0])
            y_real_07_series=pd.Series(y_real_07)
            ###########evaluating the regression quality##########
            corr_ann = round(x_prediction_07_series.corr(y_real_07_series), 5)
            rmse_val= rmse(x_prediction_07[:,0],y_real_07)
            print('scatter corr',corr_ann,'scatter rmse',rmse_val)
            print(neurons1,epochs_number,learning_rate_search)
            x_y_x=np.arange(0,2.5,0.1)
            x_y_y=np.arange(0,2.5,0.1)
            fig = plt.figure()
            ax = fig.add_subplot(111)
            ax.scatter(x_prediction_07[:,0],y_real_07,color='red')
            ax.plot(x_y_x,x_y_y)
            plt.legend()
            plt.xlabel(u"Predicted_Current_0.7V A")
            plt.ylabel(u"Real_Current_0.7V A")
            plt.show()
            K.clear_session()    

Using matplotlib backend: Qt5Agg
training...
Epoch 1/50
179438/179438 [==============================] - 7s 42us/step - loss: 0.7960
Epoch 2/50
179438/179438 [==============================] - 7s 41us/step - loss: 0.3053
Epoch 3/50
179438/179438 [==============================] - 7s 41us/step - loss: 0.1664
Epoch 4/50
179438/179438 [==============================] - 7s 41us/step - loss: 0.1327
Epoch 5/50
179438/179438 [==============================] - 7s 41us/step - loss: 0.1084
Epoch 6/50
179438/179438 [==============================] - 7s 41us/step - loss: 0.0875
Epoch 7/50
179438/179438 [==============================] - 7s 41us/step - loss: 0.0721
Epoch 8/50
179438/179438 [==============================] - 7s 41us/step - loss: 0.0628
Epoch 9/50
179438/179438 [==============================] - 7s 40us/step - loss: 0.0572
Epoch 10/50
179438/179438 [==============================] - 8s 42us/step - loss: 0.0532
Epoch 11/50
179438/179438 [==============================] - 7s 41us/step 

No handles with labels found to put in legend.


[1.6816251  0.8779103  0.9987315  ... 0.6430205  0.57321864 0.7196724 ]
[1.6848218 0.91152   1.017473  ... 0.6695    0.56765   0.74231  ]
scatter corr 0.99451 scatter rmse 0.041746665
50 50 5e-06
training...
Epoch 1/50
179438/179438 [==============================] - 7s 40us/step - loss: 0.1664
Epoch 2/50
179438/179438 [==============================] - 7s 41us/step - loss: 0.0408
Epoch 3/50
179438/179438 [==============================] - 8s 43us/step - loss: 0.0280
Epoch 4/50
179438/179438 [==============================] - 8s 44us/step - loss: 0.0206
Epoch 5/50
179438/179438 [==============================] - 7s 41us/step - loss: 0.0154
Epoch 6/50
179438/179438 [==============================] - 8s 44us/step - loss: 0.0119
Epoch 7/50
179438/179438 [==============================] - 7s 42us/step - loss: 0.0096
Epoch 8/50
179438/179438 [==============================] - 7s 40us/step - loss: 0.0081
Epoch 9/50
179438/179438 [==============================] - 7s 40us/step - loss: 0.0070


No handles with labels found to put in legend.


[1.6813905  0.906457   0.9875509  ... 0.6527156  0.5764519  0.72957754]
[1.6848218 0.91152   1.017473  ... 0.6695    0.56765   0.74231  ]
scatter corr 0.99823 scatter rmse 0.023798075
50 50 5e-05
training...
Epoch 1/50
179438/179438 [==============================] - 7s 40us/step - loss: 0.0361
Epoch 2/50
179438/179438 [==============================] - 7s 39us/step - loss: 0.0069
Epoch 3/50
179438/179438 [==============================] - 7s 40us/step - loss: 0.0053
Epoch 4/50
179438/179438 [==============================] - 7s 40us/step - loss: 0.0049
Epoch 5/50
179438/179438 [==============================] - 7s 40us/step - loss: 0.0046
Epoch 6/50
179438/179438 [==============================] - 8s 43us/step - loss: 0.0045
Epoch 7/50
179438/179438 [==============================] - 7s 40us/step - loss: 0.0044
Epoch 8/50
179438/179438 [==============================] - 7s 41us/step - loss: 0.0043
Epoch 9/50
179438/179438 [==============================] - 7s 40us/step - loss: 0.0042


...

[1.6836908  0.91255367 0.9863043  ... 0.67086256 0.56824386 0.7453763 ]
[1.6848218 0.91152   1.017473  ... 0.6695    0.56765   0.74231  ]
scatter corr 0.99818 scatter rmse 0.024114259
500 500 5e-05
training...
Epoch 1/500
179438/179438 [==============================] - 8s 42us/step - loss: 0.0167
Epoch 2/500
179438/179438 [==============================] - 7s 40us/step - loss: 0.0061
Epoch 3/500
179438/179438 [==============================] - 7s 40us/step - loss: 0.0052
Epoch 4/500
179438/179438 [==============================] - 7s 40us/step - loss: 0.0048
Epoch 5/500
179438/179438 [==============================] - 7s 40us/step - loss: 0.0045
Epoch 6/500
179438/179438 [==============================] - 7s 40us/step - loss: 0.0043
Epoch 7/500
179438/179438 [==============================] - 7s 40us/step - loss: 0.0042
Epoch 8/500
179438/179438 [==============================] - 7s 40us/step - loss: 0.0041
Epoch 9/500
179438/179438 [==============================] - 7s 41us/step - lo

179438/179438 [==============================] - 8s 42us/step - loss: 0.0031
Epoch 179/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0032
Epoch 180/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0032
Epoch 181/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0032
Epoch 182/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0032
Epoch 183/500
179438/179438 [==============================] - 7s 40us/step - loss: 0.0032
Epoch 184/500
179438/179438 [==============================] - 7s 40us/step - loss: 0.0032
Epoch 185/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0032
Epoch 186/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0032
Epoch 187/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0032
Epoch 188/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0032
Epoch 189/500

179438/179438 [==============================] - 7s 41us/step - loss: 0.0031
Epoch 357/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0031
Epoch 358/500
179438/179438 [==============================] - 7s 42us/step - loss: 0.0031
Epoch 359/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0031
Epoch 360/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0031
Epoch 361/500
179438/179438 [==============================] - 7s 42us/step - loss: 0.0031
Epoch 362/500
179438/179438 [==============================] - 7s 42us/step - loss: 0.0031
Epoch 363/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0031
Epoch 364/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0031
Epoch 365/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0031
Epoch 366/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0031: 0s - lo
Epoc

D:\anaconda\lib\site-packages\ipykernel_launcher.py:143: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
No handles with labels found to put in legend.


[1.6811047  0.91194457 0.99365366 ... 0.6744636  0.5699312  0.7429877 ]
[1.6848218 0.91152   1.017473  ... 0.6695    0.56765   0.74231  ]
scatter corr 0.99845 scatter rmse 0.022188468
500 500 0.0005


In [3]:
import eli5
def base_model():
    tmodel = Sequential() 
    tmodel.add(Dense(500, input_dim=11, kernel_initializer='random_normal',
                    bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0))) 
    tmodel.add(Dropout(0))
    tmodel.add(Dense(500, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0))) 
    tmodel.add(Dropout(0))
    tmodel.add(Dense(500, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0))) 
    tmodel.add(Dropout(0))
    tmodel.add(Dense(1, input_dim=500, activation='linear'))
    adam=optimizers.Adam(lr=0.0005)
    tmodel.compile(loss='mse', optimizer=adam) 
    return tmodel
from eli5.sklearn import PermutationImportance
my_model = KerasRegressor(build_fn=base_model,nb_epoch=500, batch_size=32, verbose= False)    
my_model.fit(X_train, y_train)
perm = PermutationImportance(my_model, random_state=1).fit(X_train,y_train)
eli5.show_weights(perm)

D:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
D:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


Weight,Feature
0.6064 ± 0.0018,x9
0.5406 ± 0.0025,x3
0.2095 ± 0.0017,x10
0.1672 ± 0.0012,x6
0.1601 ± 0.0006,x4
0.1144 ± 0.0009,x2
0.1038 ± 0.0006,x0
0.0799 ± 0.0003,x8
0.0051 ± 0.0001,x5
0.0011 ± 0.0000,x1


In [5]:
neurons1=500
epochs_number=500
learning_rate_search=0.0005
regularizer=keras.regularizers.l2(0)
dropout_rate=0
batch_size_number=32
activation1='relu'
###########keras ANN model construction##########
smodel = Sequential() 
smodel.add(Dense(neurons1, input_dim=11, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
smodel.add(Dropout(dropout_rate))
smodel.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
smodel.add(Dropout(dropout_rate))
smodel.add(Dense(1, input_dim=neurons1, activation='linear'))
adam=optimizers.Adam(lr=learning_rate_search)
smodel.compile(loss='mse', optimizer=adam)
print('training...')
smodel.fit(X_train, y_train,verbose=1, batch_size=batch_size_number,epochs=epochs_number)
sresult=smodel.predict(X_test)

training...
Epoch 1/500
179438/179438 [==============================] - 8s 44us/step - loss: 0.0174
Epoch 2/500
179438/179438 [==============================] - 8s 44us/step - loss: 0.0062
Epoch 3/500
179438/179438 [==============================] - 8s 43us/step - loss: 0.0052
Epoch 4/500
179438/179438 [==============================] - 8s 44us/step - loss: 0.0047
Epoch 5/500
179438/179438 [==============================] - 8s 42us/step - loss: 0.0044
Epoch 6/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0043
Epoch 7/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0042
Epoch 8/500
179438/179438 [==============================] - 7s 40us/step - loss: 0.0041
Epoch 9/500
179438/179438 [==============================] - 7s 40us/step - loss: 0.0040
Epoch 10/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0039
Epoch 11/500
179438/179438 [==============================] - 7s 42us/step - loss: 0.0039
Epoch 1

179438/179438 [==============================] - 7s 40us/step - loss: 0.0032
Epoch 181/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0031
Epoch 182/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0032
Epoch 183/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0032
Epoch 184/500
179438/179438 [==============================] - 7s 40us/step - loss: 0.0032
Epoch 185/500
179438/179438 [==============================] - 7s 40us/step - loss: 0.0031
Epoch 186/500
179438/179438 [==============================] - 7s 40us/step - loss: 0.0031
Epoch 187/500
179438/179438 [==============================] - 7s 40us/step - loss: 0.0032
Epoch 188/500
179438/179438 [==============================] - 7s 40us/step - loss: 0.0032
Epoch 189/500
179438/179438 [==============================] - 7s 40us/step - loss: 0.0031
Epoch 190/500
179438/179438 [==============================] - 7s 40us/step - loss: 0.0031
Epoch 191/500

Epoch 270/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0031
Epoch 271/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0031
Epoch 272/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0031: 0s
Epoch 273/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0031
Epoch 274/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0031
Epoch 275/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0031
Epoch 276/500
179438/179438 [==============================] - 7s 40us/step - loss: 0.0031: 0s 
Epoch 277/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0031
Epoch 278/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0031
Epoch 279/500
179438/179438 [==============================] - 7s 40us/step - loss: 0.0031
Epoch 280/500
179438/179438 [==============================] - 7s 40us/step - los

179438/179438 [==============================] - 7s 41us/step - loss: 0.0031
Epoch 450/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0031
Epoch 451/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0031
Epoch 452/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0031
Epoch 453/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0031
Epoch 454/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0031
Epoch 455/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0031
Epoch 456/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0031
Epoch 457/500
179438/179438 [==============================] - 7s 41us/step - loss: 0.0031
Epoch 458/500
179438/179438 [==============================] - 7s 40us/step - loss: 0.0031
Epoch 459/500
179438/179438 [==============================] - 7s 40us/step - loss: 0.0031
Epoch 460/500

In [8]:
import shap
# %matplotlib
standardized_data_SHAP = standardized_data.sample(5000,random_state=1)
SHAP_INPUT=standardized_data_SHAP.iloc[:,0:11]
SHAP_OUTPUT=standardized_data_SHAP.iloc[:,13]
X_SHAP=SHAP_INPUT.values.astype(np.float32)
y_SHAP=SHAP_OUTPUT.values.astype(np.float32)
explainer = shap.DeepExplainer(smodel,X_SHAP)
cols=[ 
              'A_c',#0
                      'A_a',#1
                      'L_cl',#2
                      'i0',#3
                      'L-PEM',#4
                      'L_GDL',#5
                      'epsl_cl',#6
                      'epsp_cl',#7
                      'T_cc',#8
                      'RH_cc',#9
                      'pressure'
                        ]
shap_values = explainer.shap_values(X_SHAP)
print(type(shap_values))


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


keras is no longer supported, please use tf.keras instead.


<class 'list'>


In [14]:
shap.summary_plot(shap_values[0], SHAP_INPUT)

In [12]:
from pdpbox import pdp
def plot_pdp(model, df, feature, cluster_flag=False, nb_clusters=None, lines_flag=False):
    
    # Create the data that we will plot
    pdp_goals = pdp.pdp_isolate(model=model, dataset=df, model_features=df.columns.tolist(), feature=feature)

    # plot it
    pdp.pdp_plot(pdp_goals, feature, cluster=cluster_flag, n_cluster_centers=nb_clusters, plot_lines=lines_flag)
    plt.show()

In [ ]:
plot_pdp(smodel, raw_input, 'T_cc')
plot_pdp(smodel, raw_input, 'i0')
plot_pdp(smodel, raw_input, 'epsl_cl')
plot_pdp(smodel, raw_input, 'L_cl')
plot_pdp(smodel, raw_input, 'RH_cc')
plot_pdp(smodel, raw_input, 'pressure')
plot_pdp(smodel, raw_input, 'A_c')
plot_pdp(smodel, raw_input, 'L-PEM')
plot_pdp(smodel, raw_input, 'L_GDL')
plot_pdp(smodel, raw_input, 'A_a')
plot_pdp(smodel, raw_input, 'epsp_cl')